In [4]:
import dhlab as dh
import pandas as pd
import os

In [290]:
authors = "anne ragde/helene uri/karl ove knausgård/jo nesbø/anne holt/unni lindell/dag solstad/agnar mykle/sigurd hoel/geir gulliksen/sigrid undset/knut hamsun".split('/')
authors

['anne ragde',
 'helene uri',
 'karl ove knausgård',
 'jo nesbø',
 'anne holt',
 'unni lindell',
 'dag solstad',
 'agnar mykle',
 'sigurd hoel',
 'geir gulliksen',
 'sigrid undset',
 'knut hamsun']

In [1]:
authors = "lars christensen/vigdis hjorth/kim småge/ingvar ambjørnsen".split('/')
authors

['lars christensen', 'vigdis hjorth', 'kim småge', 'ingvar ambjørnsen']

In [5]:
def make_corpus(author):
    corpus = dh.Corpus(author = author, ddk="839*", limit = 200).frame[['dhlabid', 'urn','authors', 'title', 'year', 'langs', 'publisher','subjects','ddc','genres']]
    corpus['single'] = [len(e.split('/')) for e in corpus.authors]
    corpus['single_title'] = [len(e.split(';')) for e in corpus.title]
    corpus = corpus[(corpus.single==1) & (corpus.langs == 'nob')].drop_duplicates('title')
    corpus = corpus[corpus.single_title <= 1]
    corpus = corpus.drop(corpus[corpus.title.str.contains('Samlede')].index)
    #print(len(corpus))
    return corpus

In [6]:
corpus = pd.concat([make_corpus(a).sample(5) for a in authors])

In [7]:
corpus['lastname'] = corpus.authors.apply(lambda x: x.split(',')[0].strip())
corpus['filename'] = corpus.lastname+'_'+corpus.urn.apply(lambda x: str(x.split(':')[-1]))

In [8]:
corpus

,dhlabid,urn,authors,title,year,langs,publisher,subjects,ddc,genres,single,single_title,lastname,filename
148,100234117,URN:NBN:no-nb_digibok_2014110508077,"Christensen , Lars Saabye",Mitt danske album,2011,nob,Cappelen Damm,norsk / litteratur / lyrikk / danmark,839.82 / 839.821,poetry,1,1,Christensen,Christensen_no-nb_digibok_2014110508077
115,100468633,URN:NBN:no-nb_digibok_2010041206052,"Christensen , Lars Saabye",Falleferdig himmel : dikt,1998,nob,Cappelen,norsk / litteratur / poesi / dikt / lyrikk / ø...,839.821,fiction,1,1,Christensen,Christensen_no-nb_digibok_2010041206052
37,100206760,URN:NBN:no-nb_digibok_2014052108240,"Christensen , Lars Saabye",Bernhard Hvals forsnakkelser : roman,2011,nob,Cappelen Damm,,839.823,novel,1,1,Christensen,Christensen_no-nb_digibok_2014052108240
75,100608463,URN:NBN:no-nb_digibok_2021101148093,"Christensen , Lars Saabye",Modellen : roman,2018,nob,Cappelen Damm,Kunstnere / Kunstnarar,839.82374,novel / text,1,1,Christensen,Christensen_no-nb_digibok_2021101148093
40,100612519,URN:NBN:no-nb_digibok_2021102048032,"Christensen , Lars Saabye",Maj,2018,nob,Cappelen Damm,Familie / Familie / norsk skjønnlitteratur / r...,839.82374,novel / text,1,1,Christensen,Christensen_no-nb_digibok_2021102048032
4,100529902,URN:NBN:no-nb_digibok_2008081904025,"Hjorth , Vigdis","Takk, ganske bra : en norsk roman i tre deler",1998,nob,Cappelen,norske / romaner / 1990-tallet,839.823,fiction,1,1,Hjorth,Hjorth_no-nb_digibok_2008081904025
11,100302178,URN:NBN:no-nb_digibok_2016070708033,"Hjorth , Vigdis",Om bare : roman,2002,nob,Cappelen,norske / romaner / 2000-tallet,839.823,fiction,1,1,Hjorth,Hjorth_no-nb_digibok_2016070708033
27,100588868,URN:NBN:no-nb_digibok_2021101248035,"Hjorth , Vigdis",Drama med Hilde : roman,2018,nob,Cappelen Damm,Kvinner / Kvinner / Ekteskap og parforhold / E...,839.8238,novel / text,1,1,Hjorth,Hjorth_no-nb_digibok_2021101248035
39,100527968,URN:NBN:no-nb_digibok_2008111904076,"Hjorth , Vigdis",Hva er det med mor : roman,2001,nob,Cappelen,alkoholmisbruk / foreldre-barn / romaner,839.823,fiction,1,1,Hjorth,Hjorth_no-nb_digibok_2008111904076
6,100606942,URN:NBN:no-nb_digibok_2021070948515,"Hjorth , Vigdis",Drama med Hilde,2009,nob,Cappelen Damm,,839.823,novel,1,1,Hjorth,Hjorth_no-nb_digibok_2021070948515


In [10]:
for u in corpus.urn:
    fn = corpus.set_index('urn').loc[u].filename
    filename = os.path.join("parsed_files", f"{fn}.pqt")
    try:
        dh.POS(u, model = 'nb_core_news_lg', start_page=1, to_page=1000).pos.to_parquet(filename)
    except:
        print(f"Problem med {filename}")
    print(u, filename)

URN:NBN:no-nb_digibok_2014110508077 parsed_files/Christensen_no-nb_digibok_2014110508077.pqt
URN:NBN:no-nb_digibok_2010041206052 parsed_files/Christensen_no-nb_digibok_2010041206052.pqt
URN:NBN:no-nb_digibok_2014052108240 parsed_files/Christensen_no-nb_digibok_2014052108240.pqt
URN:NBN:no-nb_digibok_2021101148093 parsed_files/Christensen_no-nb_digibok_2021101148093.pqt
URN:NBN:no-nb_digibok_2021102048032 parsed_files/Christensen_no-nb_digibok_2021102048032.pqt
URN:NBN:no-nb_digibok_2008081904025 parsed_files/Hjorth_no-nb_digibok_2008081904025.pqt
URN:NBN:no-nb_digibok_2016070708033 parsed_files/Hjorth_no-nb_digibok_2016070708033.pqt
URN:NBN:no-nb_digibok_2021101248035 parsed_files/Hjorth_no-nb_digibok_2021101248035.pqt
URN:NBN:no-nb_digibok_2008111904076 parsed_files/Hjorth_no-nb_digibok_2008111904076.pqt
URN:NBN:no-nb_digibok_2021070948515 parsed_files/Hjorth_no-nb_digibok_2021070948515.pqt
URN:NBN:no-nb_digibok_2010062103018 parsed_files/Småge_no-nb_digibok_2010062103018.pqt
URN:NBN:

In [11]:
for u in corpus.urn:
    fn = corpus.set_index('urn').loc[u].filename
    filename = os.path.join("ner_files", f"{fn}.pqt")
    try:
        dh.NER(u, model = 'nb_core_news_lg', start_page=1, to_page=1000).ner.to_parquet(filename)
    except:
        print(f"Problem med {filename}")
    print(u, filename)

URN:NBN:no-nb_digibok_2014110508077 ner_files/Christensen_no-nb_digibok_2014110508077.pqt
URN:NBN:no-nb_digibok_2010041206052 ner_files/Christensen_no-nb_digibok_2010041206052.pqt
URN:NBN:no-nb_digibok_2014052108240 ner_files/Christensen_no-nb_digibok_2014052108240.pqt
URN:NBN:no-nb_digibok_2021101148093 ner_files/Christensen_no-nb_digibok_2021101148093.pqt
URN:NBN:no-nb_digibok_2021102048032 ner_files/Christensen_no-nb_digibok_2021102048032.pqt
URN:NBN:no-nb_digibok_2008081904025 ner_files/Hjorth_no-nb_digibok_2008081904025.pqt
URN:NBN:no-nb_digibok_2016070708033 ner_files/Hjorth_no-nb_digibok_2016070708033.pqt
URN:NBN:no-nb_digibok_2021101248035 ner_files/Hjorth_no-nb_digibok_2021101248035.pqt
URN:NBN:no-nb_digibok_2008111904076 ner_files/Hjorth_no-nb_digibok_2008111904076.pqt
URN:NBN:no-nb_digibok_2021070948515 ner_files/Hjorth_no-nb_digibok_2021070948515.pqt
URN:NBN:no-nb_digibok_2010062103018 ner_files/Småge_no-nb_digibok_2010062103018.pqt
URN:NBN:no-nb_digibok_2012112808155 ner_f